# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [2]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import re




In [7]:
# no se como hacer esta mierda
# pillar las tablas del codigo html, igual pillar todas e iterarlas?

# seleccionar la página recurso


# petición para acceder a ello
response = requests.get(url, time.sleep(10))

# ver si la petición se ha ejectuado correctamente
if response.status_code == 200:
    #transformar el html plano en uno real (estructurando y anidando)
    soup = BeautifulSoup(response.text, 'html.parser') # es más actual que solo html
    print(soup)

else:
    print("Error")
    print(response.status_code)







Error
403


In [8]:
url = 'https://ycharts.com/companies/TSLA/revenues'

# Intentar primera solicitud con un agente de usuario estándar
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
}
response = requests.get(url, headers=headers)
time.sleep(10)  # Espera opcional de 10 segundos antes de la siguiente solicitud
# Verificar si la solicitud inicial fue bloqueada
if response.status_code == 403:
    # Si se bloquea, intentar nuevamente con un agente de usuario diferente
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    time.sleep(10)  # Espera opcional de 10 segundos antes de la siguiente solicitud
# Extraer el HTML de la respuesta final
html_data = response.text
# Imprimir o manejar el HTML según sea necesario
print(html_data)









<!doctype html>
<html lang="en" ng-jq>
    <head>
        <title>
        Tesla Revenue (Quarterly) Analysis | YCharts
        </title>

        <meta name="viewport" content="width=device-width, initial-scale=1.0" />
<!-- This code allows us to correctly display the page on mobile devices -->
<script defer>
    var viewport = document.querySelector('meta[name=viewport]');
    if (viewport) {
        var minWidth = 700;
        if (screen.width < minWidth) {
            document.head.removeChild(viewport);
            var newViewport = document.createElement('meta');
            newViewport.setAttribute('name', 'viewport');
            newViewport.setAttribute('content', 'width=' + minWidth);
            document.head.appendChild(newViewport);
        }
    }
</script>


        
        <script>
    if (navigator.userAgent.indexOf('Trident') !== -1 || navigator.userAgent.indexOf('MSIE') !== -1) {
        window.onload=function() {
            var ie_banner = document.createEle

<div>

<div class="row">
    <div class="col-6">
        <table class="table">
            <thead>
                <tr>
                    <th>Date</th>
                    <th class="text-right">Value</th>
                </tr>
            </thead>
            <tbody>
                
                    <tr>
                        <td>March 31, 2024</td>
                        <td class="text-right">
                            
                                21.30B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>December 31, 2023</td>
                        <td class="text-right">
                            
                                25.17B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>September 30, 2023</td>
                        <td class="text-right">
                            
                                23.35B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>June 30, 2023</td>
                        <td class="text-right">
                            
                                24.93B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>March 31, 2023</td>
                        <td class="text-right">
                            
                                23.33B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>December 31, 2022</td>
                        <td class="text-right">
                            
                                24.32B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>September 30, 2022</td>
                        <td class="text-right">
                            
                                21.45B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>June 30, 2022</td>
                        <td class="text-right">
                            
                                16.93B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>March 31, 2022</td>
                        <td class="text-right">
                            
                                18.76B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>December 31, 2021</td>
                        <td class="text-right">
                            
                                17.72B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>September 30, 2021</td>
                        <td class="text-right">
                            
                                13.76B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>June 30, 2021</td>
                        <td class="text-right">
                            
                                11.96B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>March 31, 2021</td>
                        <td class="text-right">
                            
                                10.39B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>December 31, 2020</td>
                        <td class="text-right">
                            
                                10.74B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>September 30, 2020</td>
                        <td class="text-right">
                            
                                8.771B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>June 30, 2020</td>
                        <td class="text-right">
                            
                                6.036B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>March 31, 2020</td>
                        <td class="text-right">
                            
                                5.985B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>December 31, 2019</td>
                        <td class="text-right">
                            
                                7.384B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>September 30, 2019</td>
                        <td class="text-right">
                            
                                6.303B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>June 30, 2019</td>
                        <td class="text-right">
                            
                                6.35B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>March 31, 2019</td>
                        <td class="text-right">
                            
                                4.541B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>December 31, 2018</td>
                        <td class="text-right">
                            
                                7.226B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>September 30, 2018</td>
                        <td class="text-right">
                            
                                6.824B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>June 30, 2018</td>
                        <td class="text-right">
                            
                                4.002B
                            
                        </td>
                    </tr>
                
                    <tr>
                        <td>March 31, 2018</td>
                        <td class="text-right">
                            
                                3.409B
                            
                        </td>
                    </tr>
                
            </tbody>
        </table>
    </div>

    
        <div class="col-6">
            <table class="table">
                <thead>
                    <tr>
                        <th>Date</th>
                        <th class="text-right">Value</th>
                    </tr>
                </thead>
                <tbody>
                    
                        <tr>
                            <td>December 31, 2017</td>
                            <td class="text-right">
                                
                                    3.288B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>September 30, 2017</td>
                            <td class="text-right">
                                
                                    2.985B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>June 30, 2017</td>
                            <td class="text-right">
                                
                                    2.790B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>March 31, 2017</td>
                            <td class="text-right">
                                
                                    2.696B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>December 31, 2016</td>
                            <td class="text-right">
                                
                                    2.285B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>September 30, 2016</td>
                            <td class="text-right">
                                
                                    2.298B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>June 30, 2016</td>
                            <td class="text-right">
                                
                                    1.270B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>March 31, 2016</td>
                            <td class="text-right">
                                
                                    1.147B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>December 31, 2015</td>
                            <td class="text-right">
                                
                                    1.214B
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>September 30, 2015</td>
                            <td class="text-right">
                                
                                    936.79M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>June 30, 2015</td>
                            <td class="text-right">
                                
                                    954.98M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>March 31, 2015</td>
                            <td class="text-right">
                                
                                    939.88M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>December 31, 2014</td>
                            <td class="text-right">
                                
                                    956.66M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>September 30, 2014</td>
                            <td class="text-right">
                                
                                    851.80M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>June 30, 2014</td>
                            <td class="text-right">
                                
                                    769.35M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>March 31, 2014</td>
                            <td class="text-right">
                                
                                    620.54M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>December 31, 2013</td>
                            <td class="text-right">
                                
                                    615.22M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>September 30, 2013</td>
                            <td class="text-right">
                                
                                    431.35M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>June 30, 2013</td>
                            <td class="text-right">
                                
                                    405.14M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>March 31, 2013</td>
                            <td class="text-right">
                                
                                    561.79M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>December 31, 2012</td>
                            <td class="text-right">
                                
                                    306.33M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>September 30, 2012</td>
                            <td class="text-right">
                                
                                    50.10M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>June 30, 2012</td>
                            <td class="text-right">
                                
                                    26.65M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>March 31, 2012</td>
                            <td class="text-right">
                                
                                    30.17M
                                
                            </td>
                        </tr>
                    
                        <tr>
                            <td>December 31, 2011</td>
                            <td class="text-right">
                                
                                    39.38M
                                
                            </td>
                        </tr>
                    
                </tbody>
            </table>
        </div>
    
</div>
</div>